# 6. 評価データ分析

In [ ]:
# 本章で用いるデータセットのダウンロードをします．
from urllib.request import urlretrieve

urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "ml-100k.zip")

In [ ]:
!pip install japanize-matplotlib

In [ ]:
# 本章で用いるデータの準備
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import japanize_matplotlib
import zipfile

with zipfile.ZipFile("./ml-100k.zip") as f:
    f.extractall()

df = pd.read_csv("ml-100k/u.data",
                 sep="\t", header=None,
                 usecols=[0,1,2],
                 names=("user_id", "movie_id", "rating"))

In [ ]:
%precision 2

In [ ]:
# データの前処理
import codecs
rating = df.pivot(index="user_id",
                  columns="movie_id", values="rating")

with codecs.open("ml-100k/u.item", "r", "utf-8", errors="ignore") as f:
    items = pd.read_csv(f, sep="|", header=None)

MOVIE_DICT = list(items[1]) #後でmovie_id -> movie名に変換するために利用
MOVIE_DICT = {i+1: v for i, v in enumerate(MOVIE_DICT)}
print(f"{rating.shape[0]}行 {rating.shape[1]}列")
rating = rating.rename(columns=MOVIE_DICT) # 映画IDを実際の映画名に変換してデータを表示
rating.head()

In [ ]:
# 評点の分布を求めるコード
print(f"評点の平均: {df['rating'].mean():.2f} 点")
print(f"1点と評価された回数: {len(df[df['rating']==1])} 回")
print(f"2点と評価された回数: {len(df[df['rating']==2])} 回")
print(f"3点と評価された回数: {len(df[df['rating']==3])} 回")
print(f"4点と評価された回数: {len(df[df['rating']==4])} 回")
print(f"5点と評価された回数: {len(df[df['rating']==5])} 回")

In [ ]:
# 1人のユーザが評価した映画数のヒストグラム
plt.xlabel("評価数")
plt.ylabel("ユーザ数")
plt.hist(rating.count(axis=1), bins=50)
plt.show()

In [ ]:
# 例題をDataFrameとして作成する
data = [
{"商品1": 3.0,"商品2": 1.0,"商品3": 4.0,"商品4": 5.0, "商品5": np.nan},
{"商品1": 3.0,"商品2": 2,"商品3": 4,"商品4": 5, "商品5": 5},
{"商品1": 5,"商品2": 1,"商品3": 3,"商品4": 2, "商品5": 2},
{"商品1": 2,"商品2": 1,"商品3": 3,"商品4": 2, "商品5": 3}
]
df_ex = pd.DataFrame(data, index=["ユーザ1", "ユーザ2", "ユーザ3", "ユーザ4"])
df_ex.head()

In [ ]:
# 相関係数を計算する
corr = df_ex.loc[["ユーザ2","ユーザ3"]].corrwith(df_ex.loc["ユーザ1"],axis=1)
print(f"user1とuser2の相関係数:{corr['ユーザ2']:.2f}")
print(f"user1とuser3の相関係数:{corr['ユーザ3']:.2f}")

In [ ]:
# 各ユーザの評点の平均を求める
df_ex.mean(axis=1)

In [ ]:
target_user = 1 # 予測対象のユーザID
print(f"評価した映画数 {rating.loc[target_user,:].count()} 件")
display(rating.loc[target_user,:].dropna()) # 実際の評点を表示

In [ ]:
# 未評価の映画を10件抽出する
unrated_movies = rating.loc[target_user][rating.loc[target_user].isnull()].index
unrated_movies = unrated_movies[:10] #10件抽出
print(list(unrated_movies)) #ユーザ1が未評価の映画10件

In [ ]:
# 評価対象の映画を評価しているユーザ集合を取得する
target_movie = "Heat (1995)" # 評価対象の映画名
target_users = rating.dropna(subset=[target_movie],
                             axis=0) # 評価対象の映画名を評価済みのユーザ

In [ ]:
# 類似ユーザを抽出する．
corrs = target_users.corrwith(rating.loc[target_user], axis=1)
similar_users = corrs.sort_values(ascending=False)[:20] #相関係数の降順でユーザをランキング
similar_users.head() # 上位5名表示

In [ ]:
# 相関係数が最も高いユーザとの評価を比べる
rating.loc[[target_user, 754]].dropna(how="any", axis=1)

In [ ]:
# 評点を予測する
mean_score = rating.loc[target_user].mean() #ユーザ1の評点の平均
score = 0.0
for user_id, cor in similar_users.iteritems():
    score += cor * (rating[target_movie][user_id] - rating.loc[user_id].mean())
score = score / np.sum(similar_users) # 類似ユーザの類似度の合計で割る
score += mean_score # 最後に対象ユーザの平均評点を加える
print("ユーザID {} の平均評点 = {:.2f} 点".format(target_user, mean_score))
print("ユーザID {} の 映画 {} に対する予測 = {:.2f} 点".format(target_user, target_movie, score))

In [ ]:
# 一連の流れを関数にまとめた
def predict_score(rating, target_user_id, target_movie_id, k=20):
    mean_score = rating.loc[target_user].mean() # 予測したいユーザの評点の平均
    target_users = rating.dropna(subset=[target_movie_id], axis=0)
    corrs = target_users.corrwith(rating.loc[target_user], axis=1)
    similar_users = corrs[corrs > 0].sort_values(ascending=False)[:k]
    score = 0
    if len(similar_users) == 0: # 類似ユーザがいない場合は評点を0点とする
        return 0
    # 本書では .iteritems()をしていますが colab版では.items()を使用しています
    for user_id, cor in similar_users.items():
        score += cor * (rating[target_movie_id][user_id] - rating.loc[user_id].mean())
    score = score / np.sum(similar_users)
    score += mean_score
    return score

In [ ]:
# 残りの未評価の映画について評点を予測する
scores = {}
for movie in unrated_movies:
    score = predict_score(rating, target_user, movie)
    scores[movie] = score
for movie, score in pd.Series(scores).sort_values(ascending=False).items():
    print(f"{movie}: {score:.2f} 点")

In [ ]:
# 評点の補正
adjusted_rating = rating.apply(lambda x: x - x.mean(), axis=1) #ユーザごとに，そのユーザの評点の平均を引く

In [ ]:
# 二つの映画のコサイン類似度を求める
from sklearn.metrics.pairwise import cosine_similarity

def cos_sim(movie_a, movie_b, min_user=1):
    t = pd.DataFrame([movie_a,movie_b]).dropna(how="any", axis=1) #a,b両者とも評価しているユーザのデータだけ抽出
    if len(t.columns) <= min_user: # 共通して評価したユーザがk人以下であれば-1を返す
        return -1
    return cosine_similarity([t.iloc[0]], [t.iloc[1]])[0][0]

In [ ]:
# 評価済みの映画を求める
rated_movies = rating.loc[:, ~rating.loc[target_user].isnull()].columns # ユーザ1が評価済みの映画のみ抽出
rated_movies

In [ ]:
# 評価済みのアイテムとの類似度を求める．
cos = lambda x: cos_sim(x, adjusted_rating.loc[:, target_movie])
sim = adjusted_rating[rated_movies].apply(cos) # ユーザ1が評価済みのアイテム全てについてコサイン類似度を計算
sim.sort_values(ascending=False) # コサイン類似度順に映画を表示

In [ ]:
# アイテムベースの協調フィルタリングに基づく評点予測を行う関数

def _predict_score_by_item_based_cf(adjusted_rating, target_user, mean_score, similar_items):
    score = 0.0
    if len(similar_items) == 0:
        return 0.0
    total_sim = np.sum(similar_items)
    for movie_id, sim in similar_items.items():
        score += sim * adjusted_rating[movie_id][target_user]
    score = score / total_sim
    score += mean_score
    return score


def predict_score_by_item_based_cf(adjusted_rating, target_user, target_movie, k=5):
    cos = lambda x: cos_sim(x, adjusted_rating.loc[:, target_movie])
    rated_movies = rating.loc[:, ~rating.loc[target_user].isnull()].columns # ユーザが評価済みの映画のみ抽出
    similar_items = adjusted_rating[rated_movies].apply(cos).sort_values(ascending=False)[:k] #類似度の高い映画を抽出
    mean_score = rating.loc[target_user].mean()
    score = _predict_score_by_item_based_cf(adjusted_rating, target_user, mean_score, similar_items)
    return score

In [ ]:
target_movie

In [ ]:
score = predict_score_by_item_based_cf(adjusted_rating, target_user, target_movie, k=5)
print(score)
print("ユーザ {} の 映画 {}:に対する予測評点 = {} 点".format(target_user, target_movie,  score))

In [ ]:
scores = {}
for movie in unrated_movies:
    print(movie)
    score = predict_score_by_item_based_cf(adjusted_rating, target_user, movie)
    scores[movie] = score

for movie, score in pd.Series(scores).sort_values(ascending=False).items():
    print(f"{movie}: {score:.2f} 点")


In [ ]:
M = rating.copy()
M

In [ ]:
# 欠損値を0で埋める
M = M.fillna(0)

In [ ]:
# NMFによる行列分解
from sklearn.decomposition import NMF
nmf = NMF(n_components=20, max_iter=500, init='random', random_state=0)
U = nmf.fit_transform(M)
V = nmf.components_;

In [ ]:
# 近似された評価値行列を確認する
M_r = np.dot(U, V)
print(M_r.shape)
M_r

In [ ]:
# k番目の潜在因子に対して高い値を持つ映画を表示する
k=13
print(rating.columns[np.argsort(V[k-1,:])[::-1]][:10].tolist())

In [ ]:
# NMFに基づく評点の予測
def predict_scores_by_nmf(target_user):
    scores = M_r[target_user-1] # target_userの全映画に対する評点
    rated_movies = rating.loc[:, (rating.loc[target_user].notnull())].dropna().columns ## target_userがすでに評価している映画のリスト
    ranking = np.argsort(scores)[::-1] #予測された評点の高い映画IDのリストを求める
    results = []
    for i in ranking:
        movie_id = i + 1
        if movie_id in rated_movies: #すでに評価した映画だったらskip
            continue
        else:
            results.append((movie_id, scores[i]))
    return results

In [ ]:
# NMFで予測された評点の高い映画を表示する
scores = predict_scores_by_nmf(target_user)
for i, score in scores[:10]:
    movie_name = MOVIE_DICT[i]
    print(f"映画:{movie_name}, 評点:{score:.2f}")